In [120]:

!sudo apt install sqlite3
!cp $SQLITE_STORE_DIR/faiss_store.db .
!ls -lh faiss_store.db

counter = !cat counter.txt
counter = int(counter[0]) - 1
%env MEMORY_BANK_NAME=bank_{counter}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
-rw-r--r-- 1 ubuntu ubuntu 179M Dec  7 23:40 faiss_store.db
env: MEMORY_BANK_NAME=bank_30


In [121]:
%%bash
echo "select value from kvstore where key = 'faiss_index:v1::${MEMORY_BANK_NAME}';" \
  | sqlite3 faiss_store.db \
  > faiss_index.json


In [122]:
!ls -lh faiss_index.json

-rw-rw-r-- 1 ubuntu ubuntu 14M Dec  7 23:40 faiss_index.json


In [123]:
import json
from IPython.display import display

# Load into dict
faiss_index = json.load(open('faiss_index.json'))
display(faiss_index.keys())
display(len(faiss_index['id_by_index']))
display(len(faiss_index['chunk_by_index']))
display(len(faiss_index['faiss_index']))

# Squad: 252 so mostly all contexts are a single chunk
# Quac: has 527 chunks so about 2 chunks per context
# NarrativeQA: 274 chunks so about 2.4 chunks per context

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

274

274

14030300

In [130]:
import os
from llama_stack_client import LlamaStackClient

# Test run
memory_bank_name = os.getenv("MEMORY_BANK_NAME")

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=memory_bank_name,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content='.', document_id='109', token_count=1), Chunk(content=' face, breaking her neck. Max and Kate are met by the police moments after.', document_id='55', token_count=17), Chunk(content=' then blasts off on a course that takes it to the system’s Earth-like third planet. Upon landing the ship indicates that the voyage has ended by ejecting a small discus from its autopilot.\nRenfry attempts to understand the autopilot and to rewind the course “tape” while the three time agents explore the jungle-draped ruins of a once-great city, in which they’ve landed. After several weeks Renfry believes that he has succeeded and he attempts to launch the ship. The ship runs its course in reverse, pausing on the desert planet and the refueling station, and brings the four men back to Earth, landing on the very spot from which they had blasted off.', document_id='112', token_count=142), Chunk(content=" the Bewilderbeast. Hiccup succeeds in freeing Toothless from

Chunk(content='.', document_id='109', token_count=1)
0.6606261712014484
Chunk(content=' face, breaking her neck. Max and Kate are met by the police moments after.', document_id='55', token_count=17)
0.6388225802693065
Chunk(content=' then blasts off on a course that takes it to the system’s Earth-like third planet. Upon landing the ship indicates that the voyage has ended by ejecting a small discus from its autopilot.\nRenfry attempts to understand the autopilot and to rewind the course “tape” while the three time agents explore the jungle-draped ruins of a once-great city, in which they’ve landed. After several weeks Renfry believes that he has succeeded and he attempts to launch the ship. The ship runs its course in reverse, pausing on the desert planet and the refueling station, and brings the four men back to Earth, landing on the very spot from which they had blasted off.', document_id='112', token_count=142)
0.6383634727926043
Chunk(content=" the Bewilderbeast. Hiccup succeeds in

In [131]:
import pandas as pd

datasets = {
    "squad": "data/squad_250.parquet",
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
display(df.iloc[87]["questions"])
display(df.iloc[87]["context"])
df.head()

array(['How did Anthony (Buck Rogers) remain asleep for 492 years?',
       'What was Wilma Deering doing when she met Buck Rogers?',
       "Rogers destroyed what gang's airship?",
       'In what war did Rogers learn his battle knowledge?',
       "What did Rogers do on the day after he shot down several of the Han's airships?",
       "In which city were the records of the Han's traitorous transactions?",
       'Which gang was the traitors?',
       'Who attacked Wilma Deering and the boy when they were on air patrol?',
       'When Rogers proved himself to the Americans, what did they do?',
       'In what year did Rogers awaken from his deep slumber?',
       'What War is this story a recollection of?',
       'What company did Buck Rodgers work for in 1927?',
       'Where in Pennsylvania was Buck Rodgers investigating reports of phenomena?',
       'What was Rodgers exposed to while investigating?',
       'How many years did Rodgers remain in "sleep"?',
       'Who was the wom

' The main character and the narrator in Armageddon 2419 A.D. is Anthony Rogers, who later appears in the various comic strips, radio shows, and film serials that follow as "Buck Rogers". Rogers recounts the events of the “Second War of Independence” that precedes the first victory of Americans over Hans, in which he plays an important role.\nBorn in 1898, he was a veteran of the Great War (World War I) and was by 1927 working for the American Radioactive Gas Corporation. He was investigating reports of unusual phenomena in abandoned coal mines near Wyoming Valley in Pennsylvania. On December 15, while investigating one of the lower levels of a mine, there was a cave-in. Exposed to radioactive gas, Rogers fell into "a state of suspended animation, free from the ravages of katabolic processes, and without any apparent effect on physical or mental faculties." Rogers remained in “sleep” for 492 years.\nHe awakes in 2419 and, thinking that he has been asleep for just several hours, wanders

,context,questions,context_length
0,"""Goblin Market"" is about two close sisters, L...","[What are the sister's names?, Who lingers at ...",3356
1,"(From Conan The Warrior, ISBN 0-441-11465-2)\...","[How old is Conan?, Why does Conan head off to...",3489
2,(Note: The following synopsis was that of Emm...,"[Who is the senior clerk at James How & Sons?,...",3202
3,"A British woman, Elise Clifton-Ward (Angelina...","[Why is Elise being followed by the Police?, W...",4331
4,A group of grifters rip off their latest mark...,"[Why was one of the four grifters shot?, How d...",3017


In [132]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

3461
3461


["What are the sister's names?",
 'Who lingers at the stream? ',
 'Who died at the beginning of winter? ',
 'What is strange about the grave in the story? ',
 "What did Laura save from the goblin's fruit? "]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [133]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=memory_bank_name,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

100%|████████████████████████████████████████████████████████████████████████████| 3461/3461 [09:26<00:00,  6.11it/s]


[[55, 91, 50, 97, 12, 59, 80, 34, 11, 55],
 [71, 65, 21, 55, 3, 65, 65, 19, 79, 101],
 [106, 16, 10, 63, 16, 17, 16, 21, 49, 60],
 [23, 98, 16, 33, 63, 63, 32, 2, 74, 16],
 [0, 0, 27, 27, 70, 43, 91, 102, 20, 79],
 [0, 79, 79, 79, 0, 91, 55, 72, 55, 91],
 [0, 0, 27, 27, 91, 79, 28, 43, 28, 91],
 [34, 55, 34, 79, 55, 79, 4, 103, 90, 74],
 [0, 79, 79, 13, 28, 71, 79, 72, 52, 59],
 [0, 0, 27, 27, 91, 100, 70, 64, 43, 79],
 [55, 12, 91, 97, 80, 50, 59, 34, 31, 55],
 [43, 71, 71, 55, 79, 79, 86, 0, 20, 111],
 [0, 0, 27, 27, 43, 91, 79, 102, 100, 45],
 [91, 91, 79, 0, 0, 89, 58, 12, 100, 68],
 [0, 79, 79, 93, 55, 28, 71, 68, 52, 0],
 [0, 79, 103, 55, 74, 28, 70, 55, 91, 59],
 [0, 0, 43, 27, 102, 70, 27, 43, 79, 28],
 [0, 0, 27, 27, 79, 102, 20, 38, 97, 88],
 [0, 0, 91, 27, 79, 27, 70, 79, 28, 28],
 [0, 0, 27, 70, 79, 27, 43, 91, 52, 79],
 [0, 0, 27, 27, 106, 30, 1, 38, 78, 30],
 [55, 12, 79, 65, 71, 55, 91, 77, 64, 34],
 [0, 0, 27, 27, 91, 79, 28, 38, 70, 79],
 [92, 70, 35, 18, 35, 55, 81, 9

In [145]:
import numpy as np

top_k = 3

res = np.array(results)
res = res[:, :top_k]
res.shape

(3461, 3)

In [146]:
targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(3461,)

(3461, 1)

(3461, 3)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 0.]])

In [147]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.4577675045747857)

In [148]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.8029471251083502)

In [149]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.7523355484927284)